# [Object对象简介](https://mp.weixin.qq.com/s?__biz=MzI4Njg5MDA5NA==&mid=2247484210&idx=1&sn=9d40e2e4c72f0727c7b7925cbe314fc0&chksm=ebd74233dca0cb2560677c7dc7746bf166195d793860c41ab477431af2cf0a6004477e27b814&scene=21###wechat_redirect)
> 声明：本文都是使用JDK1.8

Java中出现什么，都可以认为它是对象(**除了**八大基本数据类型。当然了，八大基本数据类型也能**装箱**成为对象)：

- 而Object就是这些对象的最高级别的，所有的Java对象都**隐式**地继承了Object对象(不用显示写extends继承)
- 所有的Java对象都**拥有Object默认的方法。**

Object对象的方法可以归纳成这几个：

－ `registerNatives()`//底层实现、不研究
- `hashCode()`
- `equals(Object obj)`
- `clone()`
- `toString()`
- `notify()`
- `notifyAll()`
- `wait(long timeout)`//还有重载了两个
- `finalize()`

Object一共有`11`个方法，其中一个为底层的实现`registerNatives()`，其中两个`wait()`和`wait(long timeout, int nanos)`重载方法。

- 所以我们真正需要看的就是`8个`方法
还有`一个属性`：`public final native Class<?> getClass();`，返回字节码文件对象(@code Object)

# equals和hashCode方法
equals和hashCode方法可以说是面试的重点题了，配合着String可以说在面试题中**哪都有它们的存在**。

首先，我们来看看equals和hashCode在Object中**原生**的实现吧：

hashCode：
```java
public native int hashCode();
```
equals:
```java
public boolean equals(Object obj){
    return(this == obj);
}
```
看上去都非常简单：

- hashCode()由native方法底层实现了。
- equals()就直接==判断是否相等了。
想要更加清晰它们究竟是做什么的，我们来读读它的注释：

hashCode()
1. 返回对象的一个hash值，对于是散列表结构的对象(比如HashMap)是有好处的
2. 同一个对象调用的hashCode方法，返回的int应该是一致的！(如果该对象没有被修改)
3. 如果两个对象equals()不相等，hashCode()可以不相等。但是程序员发现，如果两个对象不相等，那么hashCode也不相等的话，这会提高散列表性能
4. hashCode默认是由对象的地址转换而来的，也是根据不同的对象转换成不同的哈希值（但这种实现不是Java语言要求的，所以常常重写它）

equals()
1. 判断该对象是否与指定的对象相等
2. 自反性
3. 对称性
4. 传递性
5. 一致性
6. 传入的参数为null，应该返回的是false
7. 默认的实现是：只有这两个对象的地址相等（==），那才认为这两个对象相等（返回true）
8. 无论何时重写equals()方法，都要重写hashCode的方法(以维持hashCode的约定)

根据注释我们可以**总结以下的要点：**

- 重写`equals()`方法，就必须重写`hashCode()`的方法
- `equals()`方法默认是比较对象的地址，使用的是`==`等值运算符
- `hashCode()`方法对底层是散列表的对象有提升性能的功能
- 同一个对象(如果该对象没有被修改)：那么重复调用`hashCode()`那么返回的int是相同的！
- `hashCode()`方法默认是由对象的地址转换而来的
- `equals()`方法还有5个默认的原则：
    - 自反性--->调用`equals()`返回的是true，无论这两个对象谁调用`equals()`都好，返回的都是true
    - 一致性--->只要对象没有被修改，那么多次调用还是返回对应的结果！
    - 传递性--->`x.equals(y)`和`y.equals(z)`都返回true，那么可以得出：`x.equals(z)`返回true
    - 对称性--->`x.equals(y)`和`y.equals(x)`结果应该是相等的。
    - 传入的参数为null，返回的是false
为啥说`hashCode()`以散列表为底层带来性能的提升是很容易理解的。如果hash值都不相等，那么可以直接判断该key是不相等的了！

## equals和hashCode方法重写


In [6]:
/**
*使用IDEA默认的模板生成的equals()和hashCode()
*/
@Override
public boolean equals(Object obj){
    if(this == obj) return true; //优化操作
    if(!(obj instanceof Address)) return false; //这里默认判断的是instanceof也好(也能判断getClass),看需求了
    
    Address address = (Address) obj;
    
    /**
    *只要他们相等，那么就返回true
    */
    if(provinceNo != address.provinceNo) return false;
    if(cityNo != address.cityNo) return false;
    return streetNo == address.streetNo;
}

@Override
public int hashNode(){
    int result = provinceNo;
    result = 31 * result + cityNo; //重写了equals()就要重写hashCode(),这是hashCode的规范
    result = 31 * result + streetNo; //31哈希值尽可能不一样
    return result;
}

CompilationException: 

## String实现的equals和hashCode方法
我们在初学的时候可能就听过了：String已经实现了equals和hashCode方法了。

这也就是为什么，我们可以直接使用String.equals()来**判断两个字符串**是否相等！

In [8]:
public boolean equals(Object obj){
    if(this == obj){ //优化，地址相等就一定相等
        return true;
    }
    
    if(obj instanceof String){
        String string = (String) obj;
        int n = value.length;
        
        if(n == string.value.length){
            char ch1[] = value;
            char ch2[] = string.value;
            int i = 0;
            while(n-- != 0){  //判断这两个字符串的数据是否相等
                if(ch1[i] != ch2[i])
                    return false;
                i++;
            }
            return true;
        }
    }
    return false;
}

CompilationException: 

In [ ]:
public int hashCode(){
    int h = hash;
    if(h == 0 && value.length > 0){
        char val[] = value;
        
        for(int i = 0; i < value.lenght; i++){
            h = 31 * h + value;
        }
    }
    return h;
}

# toString方法
## 原方法

In [9]:
public String toString(){
    return getClass.getName() + "@" + Integer.toHexString(hashCode());
}

CompilationException: 

## 重写方法

In [10]:
@Override
public String toString(){
    return "Address{" +
    "provinceNo=" + proviceNo +
    ", cityN=" + cityNo +
    ", streetNo=" + streetNo +
    '}';
}

CompilationException: 

# clone方法
1. 拷贝的引用和原来对象的引用一般不相等
2. 字节码文件对象一般相等(不强求)
3. 拷贝对应和原对象调用equals一般相等(不强求)
4. 通过super.clone调用
5. 通常拷贝的对象与原对象是独立的
6. 因此往往其**成员变量(如果是可变的引用)都需要拷贝一份出去**（实现完全独立）
7. 该对象没有实现Cloneable接口会抛出异常
8. 字段本身不克隆称为浅拷贝
9. Object类本身没有实现Cloneable接口，在Object上调用clone方法一样会出现异常

看了上面的注释我们可以总结以下的要点：
- clone方法用于对象的克隆，一般想要克隆出的对象是**独立**的(与原有的对象是分开的)
- 深拷贝指的是该对象的成员变量(如果是可变引用)都应该克隆一份，浅拷贝指的是成员变量没有被克隆一份
比如一个类中的Date对象，为浅拷贝。

## 讲得乱七八糟。。。